In [1]:
import os
import glob
import json 

import numpy as np
import cPickle as pkl

import seaborn as sns
import pylab as pl
import pandas as pd

import itertools

from pipeline.python.utils import natural_keys
from pipeline.python.classifications import experiment_classes as util
import cPickle as pkl


/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

def get_animalids(fov_type='zoom2px'):
    
    all_paths = glob.glob(os.path.join(rootdir, 'JC*', '20*', '*zoom2p0x*')) #sessionmeta.json'))
    included_paths = [p for p in all_paths if int(p.split(rootdir)[1].split('/')[2]) >= 20190301] #20190406]
    animalids = sorted(list(set([p.split(rootdir)[1].split('/')[1] for p in included_paths])), key=natural_keys)
    print("Found %i animals with FOV: %s" %(len(animalids),  fov_type))
    
    return animalids

class MetaData():
    def __init__(self, animalid, rootdir='/n/coxfs01/2p-data'):
        self.animalid = animalid
        self.anesthetized_session_list = []
        self.sessions = {}
    
    def update_sessions(self, fov_type='zoom2p0x'):
        blacklist = ['JC097_20190717_FOV1', 
                     'JC089_20190523_FOV1', 'JC089_20190523_FOV2',
                     'JC092_20190527_FOV1']
        # Check if anesthetized info / visual area info stored in metafile:
        meta_info_file = os.path.join(rootdir, self.animalid, 'sessionmeta.json')
        meta_info = {}
        if os.path.exists(meta_info_file):
            try:
                with open(meta_info_file, 'r') as f:
                    meta_info = json.load(f)
            except Exception as e:
                print("...creating new meta file")
                
        # Get all session for current animal:
        session_paths =  sorted(glob.glob(os.path.join(rootdir, self.animalid,  '*', 'FOV*_%s' % fov_type)), key=natural_keys)

        print("Found %i acquisitions." % len(session_paths))
        for si, session_path in enumerate(session_paths):
            session_name = os.path.split(session_path.split('/FOV')[0])[-1]
            fov_name = os.path.split(session_path)[-1]
            print("[%s]: %s - %s" % (animalid, session_name, fov_name))
            skey = '%s_%s' % (session_name, fov_name.split('_')[0])
            if '%s_%s' % (self.animalid, skey) in blacklist:
                print("... excluded dataset, skipping")
                continue

            # Update meta info if this is a new session:
            if skey not in meta_info.keys():
                user_input = raw_input('--> (%s)\n--  Was this session anesthetized? [Y/n]' % skey)
                if user_input == 'Y':
                    #self.anesthetized_session_list.append(session_name)
                    state = 'anesthetized'
                else:
                    state = 'awake'
                visual_area = raw_input('--  Enter visual area recorded: ')
                meta_info.update({skey: {'state': state,
                                        'visual_area': visual_area}})
            else:
                state = meta_info[skey]['state']
                visual_area = meta_info[skey]['visual_area']
                
            with open(meta_info_file, 'w') as f:
                json.dump(meta_info, f, sort_keys=True, indent=4)
        return meta_info
    
            
    def get_sessions(self, fov_type='zoom2p0x', session_list = [],
                     create_new=False, rootdir='/n/coxfs01/2p-data'):

        # Check if anesthetized info / visual area info stored in metafile:
        create_meta = False
        meta_info_file = os.path.join(rootdir, animalid, 'sessionmeta.json')
        if os.path.exists(meta_info_file):
            try:
                with open(meta_info_file, 'r') as f:
                    meta_info = json.load(f)
            except Exception as e:
                print("...creating new meta file")
                create_meta = True
        else:
            create_new = True
            create_meta = True
            
        if create_meta:
            meta_info = {}
        
        # Get all session for current animal:
        if len(session_list) == 0:
            session_paths =  sorted(glob.glob(os.path.join(rootdir, animalid, '*', 'FOV*_%s' % fov_type)), key=natural_keys)
        else:
            session_paths = sorted([glob.glob(os.path.join(rootdir, animalid,  '%s' % s.split('_')[0], 
                                                           '%s_%s' % (s.split('_')[1], fov_type)))[0]\
                                   for s in session_list], key=natural_keys)
        print("Found %i acquisitions." % len(session_paths))
        for si, session_path in enumerate(session_paths):
            session_name = os.path.split(session_path.split('/FOV')[0])[-1]
            fov_name = os.path.split(session_path)[-1]
            print("[%s]: %s - %s" % (animalid, session_name, fov_name))
            skey = '%s_%s' % (session_name, fov_name.split('_')[0])

            # Load session data, if exists:
            output_dir = os.path.join(rootdir, animalid, session_name, fov_name, 'summaries')
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            session_outfile = os.path.join(output_dir, 'sessiondata.pkl')                
            
            if not create_new:
                try:
                    assert os.path.exists(session_outfile), "... session object does not exist, creating new."
                    print("... loading session object...") #% (animalid, session_name))
                    with open(session_outfile, 'rb') as f:
                        S = pkl.load(f)
                        assert 'visual_area' in dir(S), "... No visual area found, creating new."
                except Exception as e:
                    print e
                    create_new = True
            
            # Update meta info if this is a new session:
            if skey not in meta_info.keys():
                user_input = raw_input('Was this session anesthetized? [Y/n]')
                if user_input == 'Y':
                    #self.anesthetized_session_list.append(session_name)
                    state = 'anesthetized'
                else:
                    state = 'awake'
                visual_area = raw_input('Enter visual area recorded: ')
                meta_info.update({skey: {'state': state,
                                        'visual_area': visual_area}})
            else:
                state = meta_info[skey]['state']
                visual_area = meta_info[skey]['visual_area']
                

            if create_new:
                print("Creating new session object...") #% (animalid, session_name))
                S = util.Session(animalid, session_name, fov_name, 
                                 visual_area=visual_area, state=state,
                                 rootdir=rootdir)
                #S.load_data(traceid=traceid, trace_type='corrected')
                # Save session data object
                with open(session_outfile, 'wb') as f:
                    pkl.dump(S, f, protocol=pkl.HIGHEST_PROTOCOL)
                
            self.sessions[skey] = S
            
            if state == 'anesthetized' and skey not in self.anesthetized_session_list:
                self.anesthetized_session_list.append(skey)
                
            with open(meta_info_file, 'w') as f:
                json.dump(meta_info, f, sort_keys=True, indent=4)
        return self.sessions
    
            
    def load_experiments(self, experiment=None, select_subset=False,
                         state=None, visual_area=None,
                         traceid='traces001', trace_type='corrected', load_raw=False,
                         responsive_thr=0.01, responsive_test='ROC',
                         receptive_field_fit='zscore0.00_no_trim',
                         update=True, get_grouped=True,
                         rootdir='/n/coxfs01/2p-data'):
        
        # Make sure output dir exists:
        if not os.path.exists(os.path.join(rootdir, 'summary_stats', 'animals')):
            os.makedirs(os.path.join(rootdir, 'summary_stats', 'animals'))
            
        #session_stats = {}
        assert len(self.sessions.keys()) > 0, "** no sessions found! **"
        for skey, sobj in self.sessions.items():
            
            if select_subset:
                # Select subset of sessions, based on visual area or state:
                if state == 'awake' and skey in self.anesthetized_session_list:
                    continue
                if state == 'anesthetized' and skey not in self.anesthetized_session_list:
                    continue
                if visual_area is not None and sobj.visual_area != visual_area:
                    continue
            
            # Do correction on experiment names for sessions before 20190511
            if experiment == 'rfs' and int(sobj.session) < 20190511:
                experiment_name = 'gratings'
            elif experiment == 'gratings' and int(sobj.session) < 20190511: 
                continue
            elif experiment == 'blobs' and sobj.animalid == 'JC078' and sobj.session == '20190426':
                continue
            else:
                experiment_name = experiment
            
            # Either load traces/labels or summary stats:
            if load_raw:
                if sobj.data.traces is None:
                    expdict = sobj.load_data(experiment=experiment_name, traceid=traceid, trace_type=trace_type)
                else:
                    expdict = sobj.experiments[experiment_name]
            else:
                expdict = sobj.get_grouped_stats(experiment_type=experiment_name,
                                  traceid=traceid, trace_type=trace_type, 
                                  responsive_test=responsive_test, responsive_thr=responsive_thr,
                                  receptive_field_fit=receptive_field_fit,
                                  update=update, get_grouped=get_grouped,
                                  rootdir=rootdir)
        
            #if expdict is not None:
            #    session_stats[skey] = expdict
            if expdict is not None:
                outfile = os.path.join(rootdir, 'summary_stats', 'animals', '%s_%s.pkl' % (self.animalid, skey))
                with open(outfile, 'wb') as f:
                    pkl.dump(expdict, f, protocol=pkl.HIGHEST_PROTOCOL)
                print("... saved session states: %s" % outfile)
            if update:
                # Load session data, if exists:
                sobj.save_session(rootdir=rootdir)
                print("... updated session object: %s" % skey)
                    
        return #session_stats
            
    
    def save_session_stats(self, outfile=None, rootdir='/n/coxfs01/2p-data'):
        #if outfile is None:
        #    outfile = os.path.join(rootdir, '%s.pkl' % self.animalid)
        
        #with open(outfile, 'wb') as f:
        #    pkl.dump(self, f, protocol=pkl.HIGHEST_PROTOCOL)
            
        for skey, sobj in self.sessions.items():
            curr_outfile = os.path.join(rootdir, '%s_%s.pkl' % (self.animalid, skey))
            with open(curr_outfile, 'wb') as f:
                pkl.dump(sobj, f, protocol=pkl.HIGHEST_PROTOCOL)
            
        print("--- saved animal data to:\n%s" % outfile)
                     
#%%

In [3]:
rootdir = '/n/coxfs01/2p-data'
traceid = 'traces001'

animalids = ['JC067', 'JC070', 'JC073',
             'JC076', 'JC078', 'JC080', 'JC083', 'JC084', 
             'JC085', 'JC090', 'JC091', 'JC097', 'JC099', 
             'JC110', 'JC111', 'JC113', 'JC117', 'JC120']


fov_type = 'zoom2p0x'



In [4]:

animalids = get_animalids(fov_type=fov_type)

for animalid in animalids:
    A = MetaData(animalid)
    slist = A.update_sessions(fov_type=fov_type)

Found 23 animals with FOV: zoom2p0x
Found 2 acquisitions.
[JC061]: 20190306 - FOV2_zoom2p0x
[JC061]: 20190306 - FOV3_zoom2p0x
Found 2 acquisitions.
[JC067]: 20190319 - FOV1_zoom2p0x
[JC067]: 20190320 - FOV1_zoom2p0x
Found 8 acquisitions.
[JC070]: 20190314 - FOV1_zoom2p0x
[JC070]: 20190314 - FOV2_zoom2p0x
[JC070]: 20190315 - FOV1_zoom2p0x
[JC070]: 20190315 - FOV2_zoom2p0x
[JC070]: 20190315 - FOV3_zoom2p0x
[JC070]: 20190316 - FOV1_zoom2p0x
[JC070]: 20190321 - FOV1_zoom2p0x
[JC070]: 20190321 - FOV2_zoom2p0x
Found 3 acquisitions.
[JC073]: 20190321 - FOV1_zoom2p0x
[JC073]: 20190322 - FOV1_zoom2p0x
[JC073]: 20190327 - FOV1_zoom2p0x
Found 14 acquisitions.
[JC076]: 20190405 - FOV1_zoom2p0x
[JC076]: 20190405 - FOV2_zoom2p0x
[JC076]: 20190406 - FOV1_zoom2p0x
[JC076]: 20190408 - FOV1_zoom2p0x
[JC076]: 20190410 - FOV1_zoom2p0x
[JC076]: 20190419 - FOV1_zoom2p0x
[JC076]: 20190420 - FOV1_zoom2p0x
[JC076]: 20190422 - FOV1_zoom2p0x
[JC076]: 20190423 - FOV1_zoom2p0x
[JC076]: 20190424 - FOV1_zoom2p0x
[JC

In [5]:
reload(util)
reload(gd)

NameError: name 'gd' is not defined

In [18]:
from pipeline.python.classifications import get_dataset_stats as gd
options = ['-t', 'traces001', '-F', fov_type]
optsE = gd.extract_options(options)
sdata_fpath = os.path.join(optsE.aggregate_dir, 'dataset_info.pkl')

sdata = gd.aggregate_session_info(traceid=optsE.traceid, trace_type=optsE.trace_type, 
                                   state=optsE.state, fov_type=optsE.fov_type, 
                                   visual_areas=optsE.visual_areas,
                                   #blacklist=optsE.blacklist, 
                                   rootdir=optsE.rootdir)

with open(sdata_fpath, 'wb') as f:
    pkl.dump(sdata, f, protocol=pkl.HIGHEST_PROTOCOL)
        
#%%


Creating new session object...
Creating new session object...
Creating new session object...
[JC084] Li - skipping
[JC111] V1 - skipping
[JC111] Lm - skipping
Creating new session object...
[JC091] V1 - skipping
Creating new session object...
Creating new session object...
Creating new session object...
... no anat
Creating new session object...
Creating new session object...
Creating new session object...
Creating new session object...
Creating new session object...
[JC085] Lm - skipping
[JC085] Li - skipping
[JC089] V1 - skipping
[JC089] Lm - skipping
Creating new session object...
[JC090] V1 - skipping
[JC090] Lm - skipping
Creating new session object...
[JC099] V1 - skipping
[JC099] Lm - skipping
Creating new session object...
Creating new session object...
Creating new session object...
[JC101] V1 - skipping
[JC101] Lm - skipping
[JC101] Li - skipping
[JC092] V1 - skipping
[JC092] Lm - skipping
Creating new session object...
Creating new session object...
Creating new session obje

In [19]:
sdata.shape

(242, 7)

In [20]:
sdata[sdata['experiment']!='blobs_pos'].shape

(241, 7)

In [12]:
new_animals = ['JC061', 'JC067', 'JC070', 'JC073']

In [21]:
sdata = sdata[sdata['experiment']!='blobs_pos']

In [23]:
with open(sdata_fpath, 'wb') as f:
    pkl.dump(sdata, f, protocol=pkl.HIGHEST_PROTOCOL)
        

In [22]:
sdata.shape

(241, 7)